In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from pyparsing import anyOpenTag, anyCloseTag
from xml.sax.saxutils import unescape as unescape
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from collections import Counter
from scipy.sparse import csr_matrix


In [2]:
infile = open("train.dat", 'r')
text = infile.readlines()
mylist1 = []
for j in (text):
   classes = j[0:2]
   mylist1.append(classes)
    
infile1 = open("test.dat", 'r')
text1 = infile1.readlines()

text2 = text + text1





In [3]:
#removing html tag
comments = []

unescape_xml_entities = lambda s: unescape(s, {"&apos;": "'", "&quot;": '"', "&nbsp;":" "})

stripper = (anyOpenTag | anyCloseTag).suppress()
for i in range(0, len(text2)):
    #print ("in loop")
    comments.append(unescape_xml_entities(stripper.transformString(text2[i])))
#print reviews1[0]


In [4]:
docs = [l.split() for l in comments]
def filterLen(docs, minlen): 
    r""" filter out terms that are too short. docs is a list of lists, each inner list is a document represented as a list of words minlen is the minimum length of the word to keep """ 
    return [ [t for t in d if len(t) >= minlen ] for d in docs ] 
docs_final = filterLen(docs, 4)



In [5]:
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [6]:
mat = build_matrix(docs_final)

csr_info(mat)


 [nrows 50000, ncols 461560, nnz 5740688]


In [7]:

# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
mat2 = csr_idf(mat, copy=True)
mat3 = csr_l2normalize(mat2, copy=True)
print("mat1:", mat[15,:20].todense(), "\n")
print("mat2:", mat2[15,:20].todense(), "\n")
print("mat3:", mat3[15,:20].todense())

('mat1:', matrix([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  2.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.]]), '\n')
('mat2:', matrix([[ 0.        ,  0.        ,  0.38686909,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  1.90179757,  0.        ,
          0.37521501,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]), '\n')
('mat3:', matrix([[ 0.        ,  0.        ,  0.00701803,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.03449969,  0.        ,
          0.00680661,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]))


In [9]:
training_mat = mat3[:25000, :]# (14805, 73433)
test_mat = mat3[25000:50000, :]

In [11]:
cosine_sim = test_mat.dot(training_mat.T)

In [ ]:
f7=open("final6.txt", "w+") 
for i in range(0,24999):
 #print("hello")
 new_array = np.squeeze(np.asarray(cosine_sim[i,:].todense()))
 sims = list(zip(mylist1,new_array))
 #print sims
 sims.sort(key=lambda x: x[1], reverse=True)
 #print sims
 top5 = sims[:100]
 counterPositive = 0
 counterNegative = 0
 for value, key in top5:
    #print value
    #print key
    if value == '-1':
     counterNegative = counterNegative + 1
    if value == '+1':
     counterPositive = counterPositive + 1
 #print counterPositive
 #print counterNegative
 if counterPositive > counterNegative :
    f7.write('+1')
    f7.write('\n')
 else:
    f7.write('-1')
    f7.write('\n')